In [1]:
import cifar10
import matplotlib.pyplot as plt

In [2]:
cifar10.data_path = "data/CIFAR-10/"

In [3]:
cifar10.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [4]:
class_names = cifar10.load_class_names()
class_names

Loading data: data/CIFAR-10/cifar-10-batches-py/batches.meta


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [5]:
images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch


In [6]:
labels_train.shape

(50000, 10)

In [7]:
images_train.shape

(50000, 32, 32, 3)

In [8]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [9]:
data_train = images_train.reshape(50000,3072)
data_test = images_test.reshape(10000,3072)

In [10]:
from sklearn.decomposition import PCA
pca=PCA()
pca.fit(data_train)

PCA()

In [11]:
pca.components_.shape

(3072, 3072)

In [12]:
k=0
total=sum(pca.explained_variance_)
currentSum=0
while currentSum/total<0.95:
    currentSum+=pca.explained_variance_[k]
    k=k+1

In [13]:
pca_t=PCA(n_components=k,whiten=True)

In [14]:
x_transform = pca_t.fit_transform(data_train)
x_transform.shape

(50000, 217)

In [15]:
x_approx = pca_t.inverse_transform(x_transform)
print(x_approx.shape)
x_approx = x_approx.reshape((50000, 32, 32, 3))

(50000, 3072)


In [16]:
x_train_pca = x_transform
x_test_pca = pca_t.transform(data_test)

In [17]:
x_train_pca.shape

(50000, 217)

In [18]:
svc = svm.SVC()
svc.fit(x_train_pca, cls_train)

SVC()

In [19]:
y_pred = svc.predict(x_test_pca)
svc_score = accuracy_score(cls_test, y_pred)
svc_score

0.5529

In [20]:
out=[]
for i in range(len(y_pred)):
    out.append(class_names[y_pred[i]])
import pandas as pd
df=pd.DataFrame(out)

In [21]:
df.to_csv("pred.csv",index = False,header = None)